In [2]:
import os
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils import data
from PIL import Image
import pickle

In [3]:
img_label_list = pickle.load(open("sample_data/cached_train.pkl", "rb"))

In [4]:
class MiniPlaces(data.Dataset):
  def __init__(self,                 
                 label_file=None,
                 num_classes=100,
                 download=False,
                 split="train",
                 transform=None):
    self.img_label_list = img_label_list
    train_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.RandomHorizontalFlip(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
    self.transform = train_transform

  def __len__(self):
        return len(self.img_label_list)
    
  def __getitem__(self, index):
        # load img and label
        img, label = self.img_label_list[index]
        # apply data augmentation
        if self.transform is not None:
            img = self.transform(img)
        return img, label
  
train_set = MiniPlaces()
train_loader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True)

In [11]:
class LeNet(nn.Module):
    def __init__(self, input_shape=(32, 32), num_classes=100):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1, padding=0)
        self.max_pool_1 = nn.MaxPool2d(kernel_size=2, padding=0, stride=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0)
        self.max_pool_2 = nn.MaxPool2d(kernel_size=2, padding=0, stride=2)
        self.fc1 = nn.Linear(16 * 5 * 5, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 100)

    def forward(self, x):
        i = 1
        shape_dict = {}
        x = F.relu(self.conv1(x))        
        x = self.max_pool_1(x)
        shape_dict[i] = list(x.shape)
        i += 1
        x = F.relu(self.conv2(x))
        x = self.max_pool_2(x)
        shape_dict[i] = list(x.shape)
        i += 1
        x = torch.flatten(x, 1)
        shape_dict[i] = list(x.shape)
        i += 1
        x = F.relu(self.fc1(x))
        shape_dict[i] = list(x.shape)
        i += 1
        x = F.relu(self.fc2(x))
        shape_dict[i] = list(x.shape)
        i += 1
        x = self.fc3(x)       
        shape_dict[i] = list(x.shape)
        i += 1 
        return x, shape_dict

model = LeNet()
# print(model)

In [12]:
training_criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

def train_model(model, train_loader, optimizer, criterion, epoch):
    model.train()
    train_loss = 0.0
    for input, target in tqdm(train_loader, total=len(train_loader)):
        optimizer.zero_grad()
        # print(input.shape)
        output, dic = model(input)
        # print(output[0])
        print(dic)
        # print(output[0])
        break
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_loader)
    # print('[Training set] Epoch: {:d}, Average loss: {:.4f}'.format(epoch+1, train_loss))
    return train_loss

train_model(model, train_loader, optimizer, training_criterion, 0)

  0%|          | 0/3125 [00:00<?, ?it/s]

{1: [32, 6, 14, 14], 2: [32, 16, 5, 5], 3: [32, 400], 4: [32, 256], 5: [32, 128], 6: [32, 100]}


0.0

In [13]:
# for parameter in model.parameters():
#     print(parameter)

# for parameter in model.parameters():
#     print(parameter)

def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

print(get_n_params(model)/10**6)

0.151324


In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
  
print(count_parameters(model))

151324
